In [9]:
import os
import pandas as pd
from google.colab import drive

# Mount Google Drive (if you're working in Colab)
drive.mount('/content/drive')

# Function to parse the text file based on the **Options:** section
def parse_questions_file(file_path):
    questions = []
    with open(file_path, 'r', encoding='utf-8', errors='replace') as file:
        content = file.read()

        # Split the content into blocks based on **Options:**
        question_blocks = content.split("**Options:**")

        for block in question_blocks[:-1]:  # Iterate through all blocks except the last one (since it won't have options)
            question = {}

            # Step 1: Look backwards to find the question text (before **Options:**)
            lines = block.strip().split('\n')

            # Find the last line before **Options:** that doesn't start with '**' or "Question" (to get the actual question text)
            question_text = next(
                (line for line in reversed(lines) if not line.startswith("**") and not line.startswith("Question")),
                'N/A'
            )

            # Step 2: Extract options from the next block (after **Options:**)
            options_block = question_blocks[question_blocks.index(block) + 1].strip().split('\n')
            options = {}
            options['A'] = next((line[3:] for line in options_block if line.startswith("A.")), 'N/A')
            options['B'] = next((line[3:] for line in options_block if line.startswith("B.")), 'N/A')
            options['C'] = next((line[3:] for line in options_block if line.startswith("C.")), 'N/A')
            options['D'] = next((line[3:] for line in options_block if line.startswith("D.")), 'N/A')

            # Step 3: Look forward in the options block to find the correct answer and explanation
            correct_answer = next((line for line in options_block if '**Correct Answer:**' in line), 'N/A').replace('**Correct Answer:** ', '')
            explanation = next((line for line in options_block if '**Explanation:**' in line), 'N/A').replace('**Explanation:** ', '')

            # Step 4: Add all extracted data to the question dictionary
            question['Question'] = question_text.strip()
            question['Option A'] = options['A'].strip()
            question['Option B'] = options['B'].strip()
            question['Option C'] = options['C'].strip()
            question['Option D'] = options['D'].strip()
            question['Correct Answer'] = correct_answer.strip()
            question['Explanation'] = explanation.strip()

            # Add the question to the list
            questions.append(question)

    return questions

# Directory containing the question text files in Google Drive
directory = '/content/drive/My Drive'  # Adjust this path

# List to hold all questions data
all_questions = []

# Loop through all text files in the directory
for filename in os.listdir(directory):
    if filename.startswith('questio'):  # Adjust based on file naming pattern
        file_path = os.path.join(directory, filename)
        all_questions.extend(parse_questions_file(file_path))

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(all_questions)

# Save the DataFrame to an Excel file in Google Drive as .xlsx
output_file = '/content/drive/My Drive/final_questions_based_on_options.xlsx'
df.to_excel(output_file, index=False)

print(f'All questions have been combined and saved to {output_file}.')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
All questions have been combined and saved to /content/drive/My Drive/final_questions_based_on_options.xlsx.


In [10]:
import pandas as pd

# Load the existing Excel file into a DataFrame
file_path = '/content/drive/My Drive/final_questions_based_on_options.xlsx'  # Adjust the path to your file
df = pd.read_excel(file_path)

# Remove rows that contain 'N/A' in any column
df_cleaned = df.replace('N/A', pd.NA).dropna(how='any')

# Remove fully empty rows (if any)
df_cleaned = df_cleaned.dropna(how='all')

# Save the cleaned DataFrame back to a new Excel file
cleaned_file_path = '/content/drive/My Drive/cleaned_questions_output.xlsx'  # Adjust the path if needed
df_cleaned.to_excel(cleaned_file_path, index=False)

print(f"Cleaned data has been saved to {cleaned_file_path}")


Cleaned data has been saved to /content/drive/My Drive/cleaned_questions_output.xlsx
